In [34]:
# !jupyter nbconvert --to script AFL.ipynb

In [35]:
# !pip install pyomo
# !pip install --upgrade --user ortools
# !apt-get install -y glpk-utils
# !sudo apt-get update -y
# !sudo apt-get install -y coinor-cbc

In [36]:
# !pip install -U git+https://github.com/coin-or/pulp
# !sudo pulptest
# !pip install pandas

In [49]:
import numpy as np
import pandas as pd
from pulp import *

In [71]:
def test1(test_df):
    assert len(test_df[test_df['Name'].isna()])==0
    for r in test_df['Round'].unique():
        names = test_df[test_df['Round']==r]['Name'].unique()
        try:    
            assert len(test_df[~(test_df['Name'].isin(names))])==0
        except AssertionError:
            # print(r)
            # print(test_df[~(test_df['Name'].isin(names))])
            assert len(test_df[~(test_df['Name'].isin(names))])==0

In [81]:
def getPlayers(full_player_scores = pd.read_csv('full_players.csv', index_col=0)):
    full_player_scores = pd.read_csv('full_players.csv')
    full_player_scores = full_player_scores.dropna(subset=['Name'])
    full_player_scores = full_player_scores.reset_index(drop=True)
    full_player_scores['Score'] = pd.to_numeric(full_player_scores['Score'], errors='coerce')

    # testfilter
    full_player_scores = full_player_scores[full_player_scores['Round']<3]

    full_player_scores["Value"] = full_player_scores['Score']/full_player_scores['Price']
    players_to_be_kept = full_player_scores.groupby('Name').mean()[['Score','Value']]

    players_to_keep = []
    num_players_to_keep = int(len(players_to_be_kept)*0.2)
    players_to_keep += list(players_to_be_kept.sort_values('Score', ascending = False).index[:num_players_to_keep])
    players_to_keep += list(players_to_be_kept.sort_values('Value', ascending = False).index[:num_players_to_keep])

    full_player_scores = full_player_scores[full_player_scores['Name'].isin(players_to_keep)]

    full_player_scores_2021 = full_player_scores[full_player_scores['Year']==2021]
    full_player_scores = full_player_scores[full_player_scores['Year']==2020]

    # Only keep players who are playing this season
    full_player_scores = full_player_scores[full_player_scores['Name'].isin(full_player_scores_2021['Name'].unique())].dropna(how='all').dropna(how='all')

    df = full_player_scores.copy()

    df1 = df[['Name','Team','Position','Price', 'Team 1','Year']].drop_duplicates()
    df2 = df[['Round']].drop_duplicates()
    df1['key'] = 0
    df2['key'] = 0
    df = df1.merge(df2, how='outer', on=['key'])
    df = df.sort_values('Round')
    df = df.drop('key', axis=1)
    full_player_scores = full_player_scores.merge(df, on=['Name', 'Team', 'Position','Round','Price', 'Team 1','Year'],how='right').reset_index(drop=True)
    try:
        test1(full_player_scores)
    except AssertionError:
        print('WARNING: failed a test')


    player_num_games_played = full_player_scores[full_player_scores['Score']>0].groupby('Name')['Team 2'].transform('count')

    opponent_average_score = full_player_scores.groupby('Team 2')['Score'].transform('mean')
    player_average_score = full_player_scores.groupby('Name')['Score'].transform('mean')
    player_average_value = player_average_score/full_player_scores['Price']

    na_locs = full_player_scores['Score'].isna()

    full_player_scores.loc[na_locs,'Score'] = ((opponent_average_score+player_average_score)/2).loc[na_locs]

    na_locs = full_player_scores['Score'].isna()

    full_player_scores.loc[na_locs,'Score'] = (player_average_score).loc[na_locs]

    RU_Players = full_player_scores.copy()

    assert list(full_player_scores['Year'].unique())==[2020]

    RU_Players = RU_Players.sort_values('Score',ascending=False)

    RU_Players = full_player_scores.copy()

    assert list(full_player_scores['Year'].unique())==[2020]

    RU_Players = RU_Players.sort_values('Score',ascending=False)
    RU_Players['id'] = "R:"+RU_Players['Round'].astype(str)+"_P:"+RU_Players['Name']

    # Only take players who played more than 5 games
    # players_to_keep = player_num_games_played[player_num_games_played>5].index.values
    # RU_Players = RU_Players.reindex(index=players_to_keep).dropna(how='all')

    try:
        test1(RU_Players)
    except AssertionError:
        print('WARNING: failed a test')

    RU_Players.groupby(['Round','Position']).count()
    return RU_Players
RU_Players = getPlayers()

In [84]:
def getProblem(RU_Players):
  player_contraints = {}
  prob = LpProblem("aflProblem", LpMaximize)
  overall_score = LpVariable('OverallScore',0)
  for p_id,score in zip(RU_Players['id'].iteritems(),RU_Players['Score'].iteritems()):
    p_id = p_id[1]
    player_contraints[p_id] = LpVariable(p_id, 0, 1, cat='Binary')

  player_contraints = LpVariable.dicts("PC", player_contraints, 0, 1, cat='Binary')
  prob += lpSum([player_contraints[p_id[1]]*score[1] for p_id,score in zip(RU_Players['id'].iteritems(),RU_Players['Score'].iteritems())]), "Total score is maximized"

  # START TRANFERS
  RU_Players_prev_round = RU_Players
  RU_Players_prev_round['Round_prev'] = RU_Players_prev_round['Round'] - 1

  RU_transfers = RU_Players[['id','Position','Round','Name']].merge(RU_Players_prev_round[['id','Position','Round_prev','Name']],left_on=['Position','Round'],right_on=['Position','Round_prev'], suffixes = ('_prev','_next'))

  RU_transfers['Transfer'] = RU_transfers['id_prev']+'-'+RU_transfers['id_next']
  RU_transfers=RU_transfers[['id_prev',	'Round',	'Name_prev',	'id_next',	'Round_prev',	'Name_next',	'Transfer']].drop_duplicates()

  transfer_contraints = {}

  # Define transfer itermediaries
  for i,t_id in RU_transfers['Transfer'].iteritems():
    transfer_contraints[t_id] = LpVariable(t_id, 0, 1, cat='Binary')

  transfer_contraints = LpVariable.dicts("TC", transfer_contraints, 0, 1, cat='Binary')

  # map rounds end to itermediary
  for prev_player,trans in RU_transfers.groupby(['id_prev'])['Transfer'].apply(list).iteritems():
    prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[prev_player], f"Previous player equals transfer intermediatary for {prev_player} equals {trans}"

  # map intermediary to next round
  for next_player,trans in RU_transfers.groupby(['id_next'])['Transfer'].apply(list).iteritems():
    prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[next_player], f"Next player equals transfer intermediatary for {next_player} equals {trans}"

  for round, transfers in RU_transfers[RU_transfers['Name_prev'] != RU_transfers['Name_next']][['Round','Transfer']].drop_duplicates().groupby(['Round'])['Transfer'].apply(list).iteritems():
    prob += lpSum([transfer_contraints[t_id] for t_id in transfers]) <= 4, f"Round: {round}, has less than or equal to 4 transfers"
    print(f'{[transfer_contraints[t_id] for t_id in transfers]}) <= 2')
  # END TRANFERS

  # START money contraint
  money = 13 * 10**6
  for round,player in RU_Players[['id','Price','Round']].drop_duplicates().groupby(['Round'])['id'].apply(list).iteritems():
    prob += lpSum([player_contraints[p_id]*RU_Players[RU_Players['id']==p_id]['Price'].values[0] for p_id in player]) <= money, f"Round: {round}, has less than ${money}"
  # END money contraint

  #START Contrain each player to a single position per round
  RU_Players_Position_Casting = RU_Players.copy()
  RU_Players_Position_Casting['pl_pos_id'] = (RU_Players['id']+'_'+RU_Players['Position'])

  Players_Position_Casting_Series = RU_Players_Position_Casting[['id','pl_pos_id','Round']].drop_duplicates().groupby(['id','Round'])['pl_pos_id'].apply(list).iteritems()

  Players_Position_Casting = {}
  for i,r in RU_Players_Position_Casting[['pl_pos_id']].drop_duplicates().iterrows():
    Players_Position_Casting[r['pl_pos_id']] = LpVariable(r['pl_pos_id'], 0, 1, cat='Binary')

  Players_Position_Casting = LpVariable.dicts("Players_Position_Casting", Players_Position_Casting, 0, 1, cat='Binary')

  for (p_id,round), pl_pos_ids in Players_Position_Casting_Series:
      player_possible_positions = [Players_Position_Casting[pl_pos_id] for pl_pos_id  in pl_pos_ids] 
      prob += lpSum(player_possible_positions) <= player_contraints[p_id], f"Must have only have equal to {p_id} (0 or 1) of {player_possible_positions} positions in round {round}"
  #END Contrain each player to a single position per round

  # START max players from each position
  # allowed_holds_per_position = {'DE': 8, "MI" : 10, 'RU' : 3, 'FO':9}
  # allowed_holds_per_position = {'DE': 5, "MI" : 1, 'RU' : 1, 'FO':1}
  allowed_holds_per_position = {'DE': 1, "MI" : 0, 'RU' : 0, 'FO':0}

  print('START max players from each position')
  for (position,round), pl_pos_ids in RU_Players_Position_Casting[['pl_pos_id','Position','Round']].drop_duplicates().groupby(['Position', 'Round'])['pl_pos_id'].apply(list).iteritems():
    prob += lpSum([Players_Position_Casting[pl_pos_id] for pl_pos_id in pl_pos_ids]) == allowed_holds_per_position[position], f"Position: {position}, has less than {allowed_holds_per_position[position]} in round {round}"
    print('\n')
  # END max players from each position

  print('done with pre-work')
  return prob, transfer_contraints
prob, transfer_contraints = getProblem(RU_Players)

, TC_R:1_P:Sam_Reid_R:2_P:Robbie_Gray, TC_R:1_P:Sam_Reid_R:2_P:Mitch_McGovern, TC_R:1_P:Sam_Reid_R:2_P:Zac_Fisher, TC_R:1_P:Sam_Reid_R:2_P:Darcy_Cameron, TC_R:1_P:Sam_Reid_R:2_P:Jack_Martin, TC_R:1_P:Sam_Reid_R:2_P:Luke_Dahlhaus, TC_R:1_P:Sam_Reid_R:2_P:Zac_Langdon, TC_R:1_P:Sam_Reid_R:2_P:Tom_Scully, TC_R:1_P:Sam_Reid_R:2_P:Jayden_Hunt, TC_R:1_P:Sam_Reid_R:2_P:Isaac_Smith, TC_R:1_P:Sam_Reid_R:2_P:Paddy_Dow, TC_R:1_P:Sam_Reid_R:2_P:Jack_Darling, TC_R:1_P:Sam_Reid_R:2_P:Isaac_Heeney, TC_R:1_P:Sam_Reid_R:2_P:Orazio_Fantasia, TC_R:1_P:Sam_Reid_R:2_P:Darcy_Macpherson, TC_R:1_P:Sam_Reid_R:2_P:Mitchell_Wallis, TC_R:1_P:Sam_Reid_R:2_P:Shaun_Burgoyne, TC_R:1_P:Sam_Reid_R:2_P:Jake_Stringer, TC_R:1_P:Sam_Reid_R:2_P:Bailey_Banfield, TC_R:1_P:Sam_Reid_R:2_P:Alex_Neal_Bullen, TC_R:1_P:Sam_Reid_R:2_P:Eddie_Betts, TC_R:1_P:Sam_Reid_R:2_P:Travis_Colyer, TC_R:1_P:Sam_Reid_R:2_P:Oliver_Hanrahan, TC_R:1_P:Sam_Reid_R:2_P:Nathan_Jones, TC_R:1_P:Sam_Reid_R:2_P:Toby_Greene, TC_R:1_P:Sam_Reid_R:2_P:Jake_Melks

In [85]:
try:
    solver = getSolver('COIN_CMD', timeLimit=200, msg=True,gapRel = 0.2)
    prob.solve(solver)
except:
    print('USING INBUILT SOLVER!!  \n')
    solver = getSolver('PULP_CBC_CMD')
    prob.solve(solver)

USING INBUILT SOLVER!!  



PulpSolverError: Pulp: Error while executing /Users/conrad/miniconda3/envs/conrad/lib/python3.6/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc

In [79]:
prob

y_R:3_P:Martin_Gleeson <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Mason_Redman <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Michael_Hartley <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Michael_Hibberd <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Michael_Hurley <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Nathan_Wilson <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Neville_Jetta <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Nic_Newman <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Nicholas_Coffield <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Nick_Haynes <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Nick_Vlastuin <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Noah_Answerth <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Oleg_Markov <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Orazio_Fantasia <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Phil_Davis <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Reece_Conca <= 1 Integer
0 <= TC_R:2_P:Zaine_Cordy_R:3_P:Riley_Bonner <= 1 Integer
0 <= TC_R:2_P:Zaine_Cor

In [43]:
for i,v in transfer_contraints.items():
    if v.value() != 0:
        # print(v.value())
        # print(i)
        last_name = i[i.find(' ')+1: i.find('-')]
        last_letters = i[-len(last_name):]
        # print(last_name)
        # print(last_letters)
        # print
        if not last_name == last_letters:
            print(i)

R:1_P:Sam Powell-Pepper-R:2_P:Sam Powell-Pepper
R:2_P:Sam Powell-Pepper-R:3_P:Sam Powell-Pepper


In [30]:
results = []
for name, player_position in Players_Position_Casting.items():
    if player_position.value() != 0:
        results.append([name,1])
Players_selected = pd.DataFrame(results, columns=['pl_pos_id', 'is_selected'])
RU_Players_Position_Casting = RU_Players_Position_Casting.merge(Players_selected, on = ['pl_pos_id'])
RU_Players_Position_Casting.to_csv('solution.csv')
print(RU_Players_Position_Casting.groupby(['Round','Position']).sum()['is_selected'])

Round  Position
1      RU          1
2      RU          1
3      RU          1
Name: is_selected, dtype: int64


In [31]:
RU_Players_Position_Casting.sort_values(['Round'])

,Round,Team 1,Team 2,Year,Score,Name,Team,Position,Price,id,Round_prev,pl_pos_id,is_selected
0,1,Magpies,Bulldogs,2020,114.0,Brodie Grundy,Magpies,RU,867000,R:1_P:Brodie Grundy,0,R:1_P:Brodie Grundy_RU,1
1,2,Magpies,Tigers,2020,82.0,Brodie Grundy,Magpies,RU,867000,R:2_P:Brodie Grundy,1,R:2_P:Brodie Grundy_RU,1
2,3,Magpies,Saints,2020,99.0,Brodie Grundy,Magpies,RU,867000,R:3_P:Brodie Grundy,2,R:3_P:Brodie Grundy_RU,1


In [32]:
for round in RU_Players_Position_Casting['Round'].unique():
    assert RU_Players_Position_Casting[RU_Players_Position_Casting['Round']==round]['is_selected'].sum()